# Exploratory Data Analysis

To get started, we will prototype the workflow locally.

**Warning:** this notebook may fail if your local machine does not have sufficient resources. 

## Install requirements

Install required packages.

In [ ]:
!pip install --upgrade dask distributed bokeh fastparquet adlfs xgboost pandas

## Get Data

The data is modified from a Kaggle competition and hosted publicly.

start a distributed Client

In [1]:
from distributed import Client

c = Client()
c

Client Scheduler: tcp://127.0.0.1:49636 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 3 Cores: 6 Memory: 68.72 GB


initialize the Pythonic filesystem

**Tip:** if you're not using public data, you need to provide data credentials. These can be retrieved through Azure ML Datastores, e.g.:

```python
from azureml.core import Workspace

ws = Workspace.from_config()
ds = ws.get_default_datastore() # ws.datastores["my-datastore-name"]

storage_options = {
    "account_name": ds.account_name,
    "account_key": ds.account_key
}
```

In [2]:
from adlfs import AzureBlobFileSystem

container_name = "malware"
storage_options = {"account_name": "azuremlexamples"}

fs = AzureBlobFileSystem(**storage_options)
fs

list the processed (partitioned) files

In [3]:
files = fs.ls(f"{container_name}/processed")
files

['malware/processed/test.parquet/', 'malware/processed/train.parquet/']

read data into a (dask) dataframe - note pandas also accepts the ``storage_options`` argument

In [4]:
import dask.dataframe as dd

for f in files:
    if "train" in f:
        df_train = dd.read_parquet(f"az://{f}", storage_options=storage_options)
    elif "test" in f:
        df_test = dd.read_parquet(f"az://{f}", storage_options=storage_options)

df_train

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_ProcessorClass,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
npartitions=256,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,int64,float64,int64,float64,float64,float64,float64,int64,int64,float64,float64,float64,int64,object,object,object,int64,int64,object,object,object,float64,int64,object,float64,float64,object,float64,float64,object,object,float64,float64,float64,float64,float64,object,float64,object,float64,int64,float64,object,float64,float64,float64,object,object,float64,object,object,object,int64,int64,object,object,object,float64,int64,object,int64,object,object,float64,float64,object,float64,float64,float64,int64,float64,float64,int64,int64,float64,float64,float64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


## Test stuff 

In [5]:
import xgboost as xgb 

def train_a_model(df, params, num_boost_round=10):
    # load in dataframe
    df = df.compute() 

    # throw out non-numeric columns [insert real data prep here]
    cols = [col for col in df.columns if df.dtypes[col] != "object"]
    data = df[cols].drop("HasDetections", axis=1).values
    label = df["HasDetections"].values

    # train xgboost
    dtrain = xgb.DMatrix(data, label=label)
    model = xgb.train(params, dtrain, num_boost_round=num_boost_round)

    return model

In [6]:
num_boost_round = 10
params = {
    "objective": "binary:logistic",
    "learning_rate": 0.1,
    "gamma": 0,
    "max_depth": 8,
}

In [7]:
model = train_a_model(df_train.partitions[0], params, num_boost_round=num_boost_round)
model

[16:25:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [8]:
import dask 

models = [c.submit(train_a_model, df, params, num_boost_round=num_boost_round) for df in df_train.partitions]
models

[<Future: pending, key: train_a_model-84139361f42823467b419618588626d6>,
 <Future: pending, key: train_a_model-55a828f5bdb3f3f3b8828a87e5d294a4>,
 <Future: pending, key: train_a_model-22454eddd571a4a404a8b5c313db565c>,
 <Future: pending, key: train_a_model-074dae329de519db335d0f0867cd785f>,
 <Future: pending, key: train_a_model-f1737f48a2e65e7886a5e31d08917093>,
 <Future: pending, key: train_a_model-d7eef2fb730f9d1b3fa5a750314d5453>,
 <Future: pending, key: train_a_model-b0c872f9f62b773f539b0e14a469eb2e>,
 <Future: pending, key: train_a_model-3120c56cc49906d2a00b374469d6bc1e>,
 <Future: pending, key: train_a_model-6964d7fd4dddd3f44d4dc8607304df8f>,
 <Future: pending, key: train_a_model-31453a0623b3b522a1eb16ec370c7d95>,
 <Future: pending, key: train_a_model-5d85b2b85321fdb3963ad34d31e765e0>,
 <Future: pending, key: train_a_model-f7a6c5526988f2e69a4b243888ea425e>,
 <Future: pending, key: train_a_model-8bb65470e85d272429a0a3cd9d21c9fa>,
 <Future: pending, key: train_a_model-bad42a189a4d6

In [9]:
models[0]

<Future: pending, key: train_a_model-84139361f42823467b419618588626d6>